<a href="https://colab.research.google.com/github/kyochanpy/forest_bears/blob/main/make_df/funcs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 素のデータから日時のデータを生成する関数
def get_date_time_features(input_df):
    input_df_ = input_df.copy()

    no_values = input_df_["No."].values
    len_list_ = []
    for i in range(len(no_values)+1):
        if no_values[i+1] - no_values[i] != 1:
            len_list_.append(i)
            if len(len_list_) == 4:
                break
    len_list_.append(len(no_values))
    
    len_list = []
    for i in range(0,len(len_list_)):
        i_ = len_list_[i] - len_list_[i-1]
        len_list.append(i_)
    len_list[0] = len_list_[0]

    fiscal_year_list = []
    for i, j in zip(
        len_list,
        ["28", "29", "30", "1", "2"]
    ):
        fiscal_year_list = fiscal_year_list + [j] * i
    
    year_list = []
    month_list = []
    day_list = []
    day_week_list = []
    time_list = []
    for i in input_df_["発生日時"].values:
        i = i.replace(" ", "")
        if len(i) > 14:
            i_list = re.findall(r"\d+", i)
            if i_list[0] == "28":
                year_list.append(f"{i_list[0]}")
            else:
                year_list.append(f"{i_list[0]}")
            month_list.append(i_list[1])
            day_list.append(i_list[2])
            day_week_list.append(i[-7])
            time_list.append(i[-5:])
        elif len(i) <= 10:
            i_list = re.findall(r"\d+", i)
            year_list.append("不明")
            month_list.append(i_list[0])
            day_list.append(i_list[1])
            day_week_list.append(i[-2])
            time_list.append("不明")
        elif (len(i) > 10) and (i[-3] == ":"):
            i_list = re.findall(r"\d+", i)
            year_list.append("不明")
            month_list.append(i_list[0])
            day_list.append(i_list[1])
            day_week_list.append(i[-7])
            time_list.append(i[-5:])
        else:
            i_list = re.findall(r"\d+", i)
            if i_list[0] == "28":
                year_list.append(f"{i_list[0]}")
            else:
                year_list.append(f"{i_list[0]}")
            month_list.append(i_list[1])
            day_list.append(i_list[2])
            day_week_list.append(i[-2])
            time_list.append("不明")

    output_df = pd.DataFrame()
    output_df["年度"] = fiscal_year_list
    output_df["年"] = year_list
    output_df["月"] = month_list
    output_df["日"] = day_list
    output_df["曜日"] = day_week_list
    output_df["時間"] = time_list
   
    fixed_year_list = []
    for f, y, m in zip(
        output_df["年度"].values,
        output_df["年"].values,
        output_df["月"].values
    ):
        if y != "不明":
            fixed_year_list.append(y)
        else:
            if 4 <= int(m) <= 12:
                fixed_year_list.append(f)
            else:
                if y == "r2":
                    fixed_year_list.append("r3")
                else:
                    fixed_year_list.append(fixed_year_list[fixed_year_list.index(y)+1])
    output_df["年"] = fixed_year_list  

    output_df["年度"] =  output_df["年度"].astype(int)
    output_df["年"] = output_df["年"].astype(int)
    output_df["月"] = output_df["月"].astype(int)
    output_df["日"] = output_df["日"].astype(int)

    return output_df

In [ ]:
# KEY_CODEを含むデータにKEY_CODEのPolygonを追加する関数
def get_geometry_features(input_df, path):
    input_df_ = input_df.copy()
    geo_df = gpd.read_file(path)

    geo_df["KEY_CODE"] = geo_df["KEY_CODE"].astype(int)

    output_df = pd.merge(input_df_, geo_df.iloc[:, [0,-1]], on="KEY_CODE")

    return output_df

In [ ]:
# 経緯度からその地点5次地域メッシュコードを追加する関数
def get_mesh5_features(input_df):
    input_df_ = input_df.copy()

    mesh5_list = []
    for lat, lng in zip(
        input_df_["lat"].to_numpy(),
        input_df_["lng"].to_numpy()
    ):
        mesh_code = ju.to_meshcode(lat, lng, 5)
        mesh5_list.append(mesh_code)
    
    output_df = input_df.copy()
    output_df["KEY_CODE"] = mesh5_list

    return output_df

In [ ]:
# 経緯度からその地点4次地域メッシュコードを追加する関数
def get_mesh4_features(input_df):
    input_df_ = input_df.copy()

    mesh4_list = []
    for lat, lng in zip(
        input_df_["lat"].to_numpy(),
        input_df_["lng"].to_numpy()
    ):
        mesh_code = ju.to_meshcode(lat, lng, 4)
        mesh4_list.append(mesh_code)
    
    output_df = input_df.copy()
    output_df["KEY_CODE"] = mesh4_list

    return output_df

In [ ]:
# KEY_CODE(5次)を含むデータからそのKEY_CODEの人口の特徴量を生成する関数
def get_population_mesh5_features(path):
    df = pd.read_table(path, sep='\s+|,', engine='python')

    output_df = df.loc[:, ("KEY_CODE", "T000876001")].drop(0)
    output_df["KEY_CODE"] = output_df["KEY_CODE"].astype(int)
    output_df.rename(columns={"T000876001":"人口"}, inplace=True)
    output_df["人口"] = output_df["人口"].astype(int)
    return output_df

In [ ]:
# KEY_CODE(4次)を含むデータからそのKEY_CODEの人口の特徴量を生成する関数
def get_population_mesh4_features(path):
    df = pd.read_table(path, sep='\s+|,', engine='python')

    output_df = df.loc[:, ("KEY_CODE", "T000847001")].drop(0)
    output_df["KEY_CODE"] = output_df["KEY_CODE"].astype(int)
    output_df.rename(columns={"T000847001":"人口"}, inplace=True)
    output_df["人口"] = output_df["人口"].astype(int)
    return output_df

In [ ]:
# DataFrameをGeoDataFrameに変換する関数
def GDF(input_df):
    input_df_ = input_df.copy()
    output_df = gpd.GeoDataFrame(input_df_)
    return output_df

In [ ]:
# メッシュの中心座標を求める関数
def min_max_coordinates(input_df):
    input_df_ = input_df.copy()

    lng_center_list = []
    lat_center_list = []
    for n in input_df_["geometry"]:
        coordinates_list = list(zip(*n.exterior.coords.xy))
        lng_list_ = []
        lat_list_ = []
        for m in coordinates_list:
            lng_list_.append(m[0])
            lat_list_.append(m[1])
        lng_mean = sum(lng_list_) / len(lng_list_)
        lng_center_list.append(lng_mean)
        lat_mean = sum(lat_list_) / len(lat_list_)
        lat_center_list.append(lat_mean)

    output_df = input_df.copy()
    output_df["lng_center"] = lng_center_list
    output_df["lat_center"] = lat_center_list
    
    return output_df

In [ ]:
# 任意の点が15角形で表された札幌市内に含まれているか判別し、内側の点だけのDataFrameを出力する関数
def get_sapporo_df(input_df):
    input_df_ = input_df.copy()

    # 札幌を表すpolygon
    poly = Polygon( *((43.023028, 140.980742),
                      (42.886595, 141.036161),
                      (42.821037, 141.090321),
                      (42.779450, 141.160855),
                      (42.790860, 141.231606),
                      (42.872928, 141.286189),
                      (42.869773, 141.334252),
                      (42.890012, 141.421411),
                      (43.021005, 141.506059),
                      (43.060363, 141.500538),
                      (43.190986, 141.401330),
                      (43.161179, 141.224740),
                      (43.142071, 141.185851),
                      (43.105577, 141.155691),
                      (43.124406, 141.071563)) ) 
    
    points = []
    for lat, lng in zip(
        input_df_["lat_center"].to_numpy(),
        input_df_["lng_center"].to_numpy()
    ):
        points.append((lat, lng))
    
    sapporo_frag = []
    for p in points:
        if poly.encloses_point(p) == True:
            sapporo_frag.append(1)
        else:
            sapporo_frag.append(0)
    
    input_df_["sapporo_frag"] = sapporo_frag
    output_df = input_df_[input_df_["sapporo_frag"] == 1][input_df.columns]

    return output_df

In [ ]:
# 実験①のためのDataFrameを生成する関数
def get_df(population_df, base_df):
    population_df_ = population_df.copy()[["KEY_CODE", "人口"]]
    base_df_ = base_df.copy().drop("人口", axis=1)

    tmp_df = pd.DataFrame()
    for i in range(2016, 2022):
        population_df_["年度"] = i
        df = base_df_[base_df_["年度"] == i]
        df_ = pd.merge(population_df_, df,  on=["KEY_CODE", "年度"], how="left")
        tmp_df = pd.concat([tmp_df, df_])

    bears_dummy = []
    for i in tmp_df["No."]:
        if i == i:  # これでiがnanかどうか判別できる
            bears_dummy.append(1)
        else:
            bears_dummy.append(0)
    tmp_df["出没ダミー"] = bears_dummy
    dummy_df = pd.get_dummies(tmp_df["年度"], prefix="年度" ,drop_first=True)

    output_df = tmp_df[["出没ダミー", "人口"]]
    output_df = pd.concat([output_df, dummy_df], axis=1)

    for i in range(2017, 2022):
        output_df[f"交差項_{i}"] = output_df["人口"] * output_df[f"年度_{i}"]

    return output_df

In [ ]:
# 1995∼2015の5年おきのデータをmergeする関数
def make_df():
    c = 0
    for i in range(1995, 2016, 5):
        if c == 0:
            KC = get_population_mesh4_features_by_years(f"/content/drive/MyDrive/forest_bears/{str(i)}_6441.txt")[["KEY_CODE"]]
            KC[f"dummy_{str(i)}"] = int(0)
            KEY_CODE = KC
            c += 1
        else:
            KC = get_population_mesh4_features_by_years(f"/content/drive/MyDrive/forest_bears/{str(i)}_6441.txt")[["KEY_CODE"]]
            KC[f"dummy_{str(i)}"] = int(0)
            KEY_CODE = pd.merge(KEY_CODE, KC, how="outer")

    output_df = get_geometry_features(KEY_CODE, "/content/drive/MyDrive/forest_bears/MESH06441_4")

    return output_df.fillna(int(1))